# Rule Unit 작성 예시

본 문서는 1차년도 6월에 개발된 RuleUnit 클래스를 활용하여 룰 유닛을 작성하고 실행하는 과정을 설명합니다.

⚠︎ 주의사항: 코드를 변경하며 작업할 때는, 사본을 생성한 후 작업해주십시오.

## 룰 유닛 작성하기
이 부분에서는 기존의 IFC 파일에 종속적이었던 실행 흐름에서 벗어나, 새로이 정의한 RuleUnit 구조에 기반하여 룰에 포함된 항목을 작성하는 방법을 다룹니다.<br><br>
본 예시에서는 KDS 24 14 21 4.6.5.1(5) 를 룰 유닛으로 작성합니다.

먼저, Rule 작성에 필요한 프로그램을 불러오기 위해 Github에서 아래의 주소가 가리키는 파일 모음을 불러옵니다. (Github Repository를 clone합니다.)

In [ ]:
!git clone https://github.com/KU-HIAI/tomok.git

Cloning into 'tomok'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 175 (delta 54), reused 85 (delta 31), pack-reused 58
Receiving objects: 100% (175/175), 451.83 KiB | 3.42 MiB/s, done.
Resolving deltas: 100% (62/62), done.


필요한 파일을 불러온 다음에는 현재 작업 경로를 tomok의 파일이 있는 위치로 이동합니다.<br><br>
아래의 코드 블록을 실행하면, 작성자의 기본 작업 경로가 필요한 파일이 있는 위치로 이동됩니다.

In [ ]:
cd tomok

/content/tomok


## 룰 유닛 작성해보기
지금까지의 내용을 바탕으로 룰 유닛을 작성해봅시다.

새로운 룰 유닛을 작성하기 위해서는, 아래 코드 블럭에서 클래스 이름, 메타정보에 해당하는 변수, 실행 함수의 이름과 내용을 수정하면 됩니다.

In [ ]:
import math
from typing import List

import tomok

# 작성하는 룰에 맞게 클래스 이름 수정 (KDS241421_04010201_03)
class KDS241421_04010201_03 (tomok.RuleUnit):

    # 아래 클래스 멤버 변수에 할당되는 값들을 작성하는 룰에 맞게 수정
    priority = 1   # 건설기준 우선순위
    author = 'Chanwoo Yang'  # 작성자명
    ref_code = 'KDS 24 14 21 4.1.2.1 (3)' # 건설기준문서
    ref_date = '2021-04-12'  # 디지털 건설문서 작성일
    doc_date = '2023-11-14'  # 건설기준문서->디지털 건설기준 변환 기준일
    title = '전단보강철근이 배치된 부재의 설계전단강도'    # 건설기준명

    #
    description = """
    콘크리트교 설계기준 (한계상태설계법)
    4. 설계
    4.1 극한한계상태
    4.1.2 전단
    4.1.2.1 일반
    (3)
    """
    # https://dillinger.io/ 표와 이미지 랜더링 확인 사이트
    # 이미지 링크 변환 사이트 https://www.somanet.xyz/2017/06/blog-post_21.html
    # 건설기준문서내용(text)
    content = """
    \n (3) 전단보강철근이 배치된 부재의 설계전단강도 $V_{d}$는 4.1.2.3에 규정된 전단보강철근의 항복을 기준으로 정한 설계전단강도 $V_{sd}$ 값으로 하여야 한다. 이때 $V_{sd}$의 값은 콘크리트 스트럿의 압축 파괴를 기준으로 정한 최대설계전단강도 $V_{d,max}$를 초과하지 않아야 한다. 즉,
    \n $$ V_{d} = V_{sd} \\leq V_{d,max} \quad (4.1-6b) $$
    """
    # 플로우차트(mermaid)
    flowchart = """
    flowchart TD
    subgraph Python_Class
    A["전단보강철근이 배치된 부재의 설계전단강도"];
    B["KDS 24 14 21 4.1.2.1 (3)"];
    A ~~~ B
    end

		subgraph Variable_def;
		VarOut1[/출력변수: 전단보강철근이 배치된 부재의 설계전단강도/];
		VarIn1[/입력변수: 설계전단강도/];
		VarIn2[/입력변수: 최대설계전단강도/];


		VarOut1~~~VarIn1 & VarIn2

		end
		Python_Class ~~~ Variable_def;
		Variable_def--->C--->D

		C["<img src='https://latex.codecogs.com/svg.image?V_{d}=V_{sd}\leq&space;V_{d,max}'>---------------------------------"]
		D(["전단보강철근이 배치된 부재의 설계전단강도"])
    """

    # 작성하는 룰에 맞게 함수 이름과 내용을 수정,
    @tomok.rule_method
    def Design_shear_strength_of_members_with_shear_rebar(fOVd,fIVsd,fIVdmax) -> float:
        """전단보강철근이 배치된 부재의 설계전단강도

        Args:
             fOVd (float): 전단보강철근이 배치된 부재의 설계전단강도
             fIVsd (float): 설계전단강도
             fIVdmax (float): 최대설계전단강도


        Returns:
            float: 콘크리트교 설계기준 (한계상태설계법) 4.1.2.1 일반 (3)의 값
        """

        fOVd = fIVsd
        if fOVd <= fIVdmax:
          return fOVd, "PASS"
        else:
          return fOVd, "Fail"

작성한 룰 유닛은 아래의 코드 블럭과 같이 생성하여, 작성자가 임의로 검증을 수행할 수 있습니다.

In [ ]:
my_RuleUnit = KDS241421_04010201_03()

In [ ]:
fOVd = None
fIVsd = 30
fIVdmax = 40

In [ ]:
Rule_Review_Result = my_RuleUnit.Design_shear_strength_of_members_with_shear_rebar(fOVd,fIVsd,fIVdmax)
# 해당건설기준 항목 의 값은?

print("RuleUnit Review Result: {}".format(Rule_Review_Result))

RuleUnit Review Result: (30, 'PASS')


<br><br>
아래의 코드를 통해, 룰 유닛의 content(건설기준 항목의 실제 내용)의 markdown 렌더링 결과를 확인할 수 있습니다.

In [ ]:
my_RuleUnit.render_markdown()


    
 (3) 전단보강철근이 배치된 부재의 설계전단강도 $V_{d}$는 4.1.2.3에 규정된 전단보강철근의 항복을 기준으로 정한 설계전단강도 $V_{sd}$ 값으로 하여야 한다. 이때 $V_{sd}$의 값은 콘크리트 스트럿의 압축 파괴를 기준으로 정한 최대설계전단강도 $V_{d,max}$를 초과하지 않아야 한다. 즉,
    
 $$ V_{d} = V_{sd} \leq V_{d,max} \quad (4.1-6b) $$
    